In [1]:
import pandas as pd
from datetime import datetime


In [2]:
col_order = ['DATE', 'OUTLET', 'AUTHOR', 'DESCRIPTION', 'TEXT']

In [3]:
youtube = pd.read_csv("Data/Youtube Comments.csv")
youtube['OUTLET'] = "YouTube"
youtube['DESCRIPTION'] = pd.NA
youtube.drop('video_id', axis=1, inplace=True)
youtube.drop('Unnamed: 0', axis=1, inplace=True)

youtube = youtube[col_order]

youtube['DATE'] = pd.to_datetime(youtube['DATE'], errors='coerce')
youtube['DATE'] = youtube['DATE'].dt.strftime('%d-%m-%Y')

In [4]:
bing = pd.read_csv("Data/bing_data.csv")
bing.drop('ARTICLE NUMBER', axis=1, inplace=True)
bing['OUTLET'] = bing['AUTHOR']
bing['AUTHOR'] = pd.NA
bing.drop('URL', axis=1, inplace=True)

bing = bing[col_order]
bing['DATE'] = pd.to_datetime(bing['DATE'], errors='coerce')
bing['DATE'] = bing['DATE'].dt.strftime('%d-%m-%Y')

In [5]:
google = pd.read_csv("Data/google_data.csv")
google['URL'] = google['URL'].str.replace('https://', '')
google['URL'] = google['URL'].str.replace('www.', '')
google['URL'] = google['URL'].str.replace(r'\.c.*', '', regex=True)
google['URL'] = google['URL'].str.replace(r'\.e.*', '', regex=True)
google['URL'] = google['URL'].str.replace(r'\.o.*', '', regex=True)
google['URL'] = google['URL'].str.replace(r'\.g.*', '', regex=True)
google['URL'] = google['URL'].str.replace(r'\.f.*', '', regex=True)
google['URL'] = google['URL'].str.replace(r'\.i.*', '', regex=True)

google['OUTLET'] = google['URL']
google['AUTHOR'] = pd.NA

google.drop('URL', axis=1, inplace=True)
google.drop('Title', axis=1, inplace=True)

col_names = ['DATE', 'DESCRIPTION', 'TEXT', 'OUTLET', 'AUTHOR' ]
google.columns = col_names

google = google[col_order]

google['DATE'] = pd.to_numeric(google['DATE'], errors='coerce')
google.dropna(subset=['DATE'], inplace=True)

google['DATE'] = pd.to_datetime(google['DATE'], errors='coerce')
google['DATE'] = google['DATE'].dt.strftime('%d-%m-%Y')

In [6]:
factivaA = pd.read_csv("Factiva.csv")
factivaA.drop('ARTICLE_NUMBER', axis=1, inplace=True)
factivaA.drop('ERRORS', axis=1, inplace=True)
factivaA['DESCRIPTION'] = pd.NA

factivaA = factivaA[col_order]

factivaA['DATE'] = pd.to_datetime(factivaA['DATE'], errors='coerce')
factivaA['DATE'] = factivaA['DATE'].dt.strftime('%d-%m-%Y')

In [7]:
def find_date_index(lst):
    for index, item in enumerate(lst):
        try:
            datetime.strptime(item, '%d %B %Y')
            return index 
        except ValueError:
            pass 
    return None 

def cleaner(file_content, code, outlet):

    articles_list = file_content.split('\nDocument ' + code)

    texts = []
    authors = []
    dates = []  

    counter = 0

    for element in articles_list:
        
        elem_list = element.split('\n')

        if counter > 0:
            try:
                elem_list = elem_list[3:]
            except IndexError:
                break

        date_loc = find_date_index(elem_list)

        if not date_loc is None:

            temp_author = elem_list[date_loc-2]
            authors.append(temp_author)

            temp_date = elem_list[date_loc]
            dates.append(temp_date)

            temp_text = '\n'.join(elem_list[date_loc+6:])
            texts.append(temp_text)
    
        else:
            continue

        final_df = pd.DataFrame({'DATE': dates, 'OUTLET': outlet, 'AUTHOR': authors, 'DESCRIPTION': pd.NA, 'TEXT': texts})

        final_df = final_df[col_order]

        final_df['DATE'] = pd.to_datetime(final_df['DATE'], format='%d %B %Y', errors='coerce')

        final_df.dropna(subset=['DATE'], inplace=True)

        final_df['DATE'] = final_df['DATE'].dt.strftime('%d-%m-%Y')

        counter = counter + 1


    return final_df
    

In [8]:
txt_files_to_clean = [('b_i_articles_cleaned.txt','BI','business insider'),
                      ('barrons_articles_cleaned.txt','BON','barrons'),
                      ('the_guardian_articles_cleaned.txt','GRD','guardian'),
                      ('w_p_articles_cleaned.txt','WP','washington post')]

factivaAlex = pd.DataFrame(columns=factivaA.columns)

for file_info in txt_files_to_clean:

    with open('Data/'+ file_info[0], 'r') as file:
        file_content = file.read()

    temp_df = cleaner(file_content, file_info[1], file_info[2])

    factivaAlex = pd.concat([factivaAlex, temp_df], ignore_index=True)



In [9]:
factivaAlex.to_csv("Factiva_Alex.csv", encoding='utf-8', index=False)

In [11]:
ArticleData = pd.concat([youtube, bing, google, factivaA, factivaAlex], ignore_index=True)
ArticleData.to_csv("ArticleData.csv", encoding='utf-8', index=False)